In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 30.6 MB/s eta 0:00:00


In [3]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pennylane as qml

# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Set the random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [43]:
# Define the path to your text file
file_path = '/content/AI_2qubits_training_data.txt1'
elements = []

# Open the file to read its contents
with open(file_path, 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

split_characters = [line.split() for line in lines][0]
for line in split_characters:
  x = [char for char in line]
  elements.append(x)
print(elements)

[['0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0'], ['0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '0', '

In [44]:
import csv

# Open a new CSV file in write mode
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each sublist to the CSV file
    for row in elements:
        writer.writerow(row)

In [58]:
class DigitsDataset(Dataset):
    """Pytorch dataloader for the Optical Recognition of Handwritten Digits Data Set"""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv_file = csv_file
        self.transform = transform
        self.df = self.filter_by_label()

    def filter_by_label(self):
        # Use pandas to return a dataframe of only zeros
        df = pd.read_csv(self.csv_file)
        return df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()


        image = self.df
        print(image)
        image = np.array(image)
        image = image.astype(np.float32).reshape(100, 100)


        if self.transform:
            image = self.transform(image)

        # Return image and label
        return image, 1

In [59]:
image_size = 100  # Height / width of the square images
batch_size = 1

transform = transforms.Compose([transforms.ToTensor()])
dataset = DigitsDataset(csv_file="/content/output.csv", transform=transform)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, drop_last=True
)


In [60]:
plt.figure(figsize=(8,2))

for i in range(8):
    image = dataset[i][0].reshape(image_size,image_size)
    plt.subplot(1,8,i+1)
    plt.axis('off')
    plt.imshow(image.numpy(), cmap='gray')

plt.show()

      0  1  0.1  0.2  1.1  1.2  1.3  1.4  1.5  1.6  ...  1.47  1.48  1.49  \
0     0  1    1    0    0    1    1    0    1    1  ...     0     1     1   
1     1  1    1    0    1    0    0    1    0    1  ...     0     1     1   
2     1  1    0    1    0    0    0    0    1    1  ...     1     1     0   
3     0  0    0    0    0    0    0    0    0    0  ...     0     0     1   
4     0  0    0    1    1    1    1    1    1    1  ...     0     0     1   
...  .. ..  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
1994  0  1    0    0    1    1    0    0    1    1  ...     1     1     0   
1995  0  1    0    1    0    0    1    1    0    1  ...     0     1     0   
1996  1  1    0    0    1    0    1    1    0    1  ...     1     0     0   
1997  0  1    0    1    1    1    1    1    0    0  ...     0     0     1   
1998  1  1    0    1    1    0    1    0    0    1  ...     0     1     0   

      1.50  1.51  1.52  0.43  0.44  1.53  0.45  
0        0     0     0    

ValueError: cannot reshape array of size 199900 into shape (100,100)

<Figure size 800x200 with 0 Axes>